In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D,MaxPool2D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.python.client import device_lib
import itertools
import os 
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
from tensorflow.python.client import device_lib
import sklearn
import wandb
from wandb.keras import WandbCallback
import glob


In [2]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
run = wandb.init(project='suml',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.0001,
                     "epochs": 10,
                     "batch_size": 10,
                     "loss_function": "categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "paintings"
                 })
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ratkac99 (use `wandb login --relogin` to force relogin)
C:\Users\Kacper\.conda\envs\test\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [4]:
train_path = 'C:/Users/Kacper/Desktop/suml_projekt/data/train'
valid_path = 'C:/Users/Kacper/Desktop/suml_projekt/data/valid'
test_path = 'C:/Users/Kacper/Desktop/suml_projekt/data/test'

#1153,4612

In [6]:
classes = []
for file in os.listdir('C:/Users/Kacper/Desktop/suml_projekt/data/train'):
    classes.append(file)


In [7]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224),classes=classes,batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224),classes=classes,batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224),classes=classes,batch_size=10,shuffle=False)


Found 5218 images belonging to 23 classes.
Found 1589 images belonging to 23 classes.
Found 1476 images belonging to 23 classes.


In [8]:
assert train_batches.n == 5218
assert valid_batches.n == 1589
assert test_batches.n == 1476
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 23

In [9]:
cm_plot_labels = classes

In [10]:
base_model = keras.applications.resnet50.ResNet50(include_top=False)

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

predictions = Dense(23, activation='softmax')(x)

model = keras.models.Model(inputs=base_model.input, outputs=predictions)

In [11]:
for layer in base_model.layers:
    layer.trainable = False

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics="accuracy")

In [14]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=10,verbose=2,callbacks=[WandbCallback(data_type="image", validation_data=valid_batches, labels=cm_plot_labels)])

Epoch 1/40


C:\Users\Kacper\.conda\envs\test\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


522/522 - 23s - loss: 1.2901 - accuracy: 0.6355 - val_loss: 1.4334 - val_accuracy: 0.5909 - 23s/epoch - 43ms/step
Epoch 2/40
522/522 - 18s - loss: 0.6059 - accuracy: 0.8266 - val_loss: 1.3173 - val_accuracy: 0.6300 - 18s/epoch - 35ms/step
Epoch 3/40
522/522 - 18s - loss: 0.3569 - accuracy: 0.9038 - val_loss: 1.2910 - val_accuracy: 0.6444 - 18s/epoch - 35ms/step
Epoch 4/40
522/522 - 18s - loss: 0.2145 - accuracy: 0.9484 - val_loss: 1.2716 - val_accuracy: 0.6595 - 18s/epoch - 34ms/step
Epoch 5/40
522/522 - 17s - loss: 0.1264 - accuracy: 0.9810 - val_loss: 1.3656 - val_accuracy: 0.6583 - 17s/epoch - 33ms/step
Epoch 6/40
522/522 - 17s - loss: 0.0735 - accuracy: 0.9943 - val_loss: 1.3432 - val_accuracy: 0.6602 - 17s/epoch - 34ms/step
Epoch 7/40
522/522 - 18s - loss: 0.0450 - accuracy: 0.9985 - val_loss: 1.4392 - val_accuracy: 0.6488 - 18s/epoch - 34ms/step
Epoch 8/40
522/522 - 17s - loss: 0.0279 - accuracy: 1.0000 - val_loss: 1.4400 - val_accuracy: 0.6595 - 17s/epoch - 33ms/step
Epoch 9/40


In [15]:
predictions = model.predict(x=test_batches, verbose=0)

In [20]:
cm = confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions,axis=-1))

In [22]:
wandb.log({
            
            "confusion_matrix" : wandb.sklearn.plot_confusion_matrix(test_batches.classes,
                                                            np.argmax(predictions,axis=-1),
                                                            cm_plot_labels)
            })

In [23]:
wandb.finish()

accuracy,▁▅▆▇█████████████▇██████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇▇▆▇█▇▇█▇▇▇██▄▇▇▇▇▇████▇▇▇▇█▇█▇█▇███
val_loss,▂▁▁▁▂▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
accuracy,1.0
best_epoch,3
best_val_loss,1.27162
epoch,39
loss,3e-05
val_accuracy,0.66457


In [24]:
model.save("fine_tunned.h5")

C:\Users\Kacper\.conda\envs\test\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
